# Make river_flux csv for fvcom-toolbox
###### author: Ishid @2022/11/16

水質水文データベースの一時間ごとの水位データを一か月ごとにtxt形式でダウンロードしたものをfvcom-toolboxで読み込めるcsvとして出力する。
ダウンロードするときに同一地点では地点名を必ずファイル名に入れること。

In [ ]:
from glob import glob
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import codecs

def make_river_level(place_list):
    """
水質水文データベースから各月ごとにdat形式でダウウンロードしたデータをfvcom-toolbox
で処理できる形のcsvに出力する。datファイル名に統一した名前を入れ、その名前をlistで
引数に入れる。
    """
    df = pd.DataFrame()
    for place in place_list:
        flux = []
        paths =glob('../'+place+'*') #datがあるpathを指定してください！
        print(paths)
        #一日のデータをresの各行で格納
        res = []
        for path in paths:
            with codecs.open(path,'r','utf-8','ignore') as f:
                l = f.readlines()
                for i in range(10,len(l)): #11行目以降が時系列データになっています。
                    res.append(l[i].split(','))
        


        #一日のデータのうち、空白や記号を取り除きます。奇数番目にデータが入っている
        data = []
        for i in range(len(res)):
            data_cld = [res[i][j].replace('    ','') for j in range(1,len(res[i]),2)]
            data.append(data_cld)
        
        #一行のlistにまとめる
        flux = [float(data[i][j]) for i in range(len(res))for j in range(24)]

        datelist = []
        for x in range((len(res))*24):
            t = dt.strptime(res[0][0], '%Y/%m/%d')+ timedelta(hours=1) \
                    +timedelta(hours=x) #初期時間からhours=xだけずらした時間
            t = t.strftime('%d-%m-%Y %H:%M') #書式の変換
            datelist.append(t)
            
        #create df
        df['datetime'] = datelist
        print(place)
        df[place] = flux
    return df

flux = make_river_level(['tamagawa'])
print(flux)

In [43]:
flux = make_river_level(['ishihara','noda','kamenoko','oashi','sugama'])
flux

['..\\ishihara202001.dat', '..\\ishihara202002.dat', '..\\ishihara202003.dat', '..\\ishihara202004.dat', '..\\ishihara202005.dat', '..\\ishihara202006.dat', '..\\ishihara202007.dat', '..\\ishihara202008.dat', '..\\ishihara202009.dat', '..\\ishihara202010.dat', '..\\ishihara202011.dat', '..\\ishihara202012.dat']
ishihara
['..\\noda202001.dat', '..\\noda202002.dat', '..\\noda202003.dat', '..\\noda202004.dat', '..\\noda202005.dat', '..\\noda202006.dat', '..\\noda202007.dat', '..\\noda202008.dat', '..\\noda202009.dat', '..\\noda202010.dat', '..\\noda202011.dat', '..\\noda202012.dat']
noda
['..\\kamenoko202001.dat', '..\\kamenoko202002.dat', '..\\kamenoko202003.dat', '..\\kamenoko202004.dat', '..\\kamenoko202005.dat', '..\\kamenoko202006.dat', '..\\kamenoko202007.dat', '..\\kamenoko202008.dat', '..\\kamenoko202009.dat', '..\\kamenoko202010.dat', '..\\kamenoko202011.dat', '..\\kamenoko202012.dat']
kamenoko
['..\\oashi202001.dat', '..\\oashi202002.dat', '..\\oashi202003.dat', '..\\oashi202004

,datetime,ishihara,noda,kamenoko,oashi,sugama
0,01-01-2020 01:00,0.57,0.22,0.23,0.39,-0.11
1,01-01-2020 02:00,0.56,0.22,0.19,0.38,-0.09
2,01-01-2020 03:00,0.56,0.22,0.17,0.38,-0.10
3,01-01-2020 04:00,0.55,0.22,0.17,0.39,-0.09
4,01-01-2020 05:00,0.55,0.22,0.16,0.39,-0.09
...,...,...,...,...,...,...
8779,31-12-2020 20:00,0.63,0.08,0.43,0.27,-0.14
8780,31-12-2020 21:00,0.64,0.09,0.26,0.27,-0.13
8781,31-12-2020 22:00,0.63,0.10,0.21,0.27,-0.14
8782,31-12-2020 23:00,0.63,0.10,0.21,0.27,-0.14


In [45]:
def level2flux(df):
    """
    https://www.pari.go.jp/search-pdf/no1276.pdf
    を元に算出・大雑把な判定が多く完ぺきとはいいがたい。チューニング前提。
    特に中川は根拠に乏しい。
    """
    df_new =pd.DataFrame()
    columns = df.columns
    if 'ishihara' in columns:
        df_new['Tamagawa'] = (-21.004*(df['ishihara']**3) +264.5*(df['ishihara']**2) -313.76*df['ishihara']+106.7)/1.76
    #大芦橋と菅間の流量の和を荒川から東京湾への流入量とした
    if 'oashi' and 'sugama'in columns:
        df_new['arakawa'] = 45.677*(df['oashi']**2)-0.2214*df['oashi']-8.03 \
                        +11.696*(df['sugama']**2)+45.795*df['sugama']+10.386 \
                        +48.2
        df_new['sumidagawa'] = 0.6*df_new['arakawa']
    if 'noda' in columns:
        df_new['edogawa'] = 0.77* (3.4568*(df['noda']**3)+5.4914*(df['noda']**2)+69.395*df['noda']+14.053)-10
        for i in range(len(df)):
            if df_new.at[i,'edogawa'] <250:
                df_new.at[i,'nakagawa'] = df_new.at[i,'edogawa']
            else:
                df_new.at[i,'nakagawa'] = 0.55 * df_new.at[i,'edogawa']
    if 'kamenoko' in columns:
        df_new['Tsurumigawa'] = 1.7 * (18.051*(df['kamenoko']**2)-18.055*df['kamenoko']+6.8849) +20.0
    
    #Q隅田川 = 0.6荒川
    #Q 江戸川 = 0.77 × (3.4568 H^3野田 + 5.4914 H^2野田+ 69.395H^野田 +14.053) -10
    #Q 中川 = Q江戸川 Q江戸川 < 250
    #Q 中川 = 0.55*Q江戸川 Q江戸川 ≥ 250
    #Q 鶴見川=1.7 * (18.051*亀^2 -18.055*亀 + 6.8449）+ 20.0
    return df_new    

Tamagawa = level2flux(flux)
Tamagawa.to_csv('river_flux2020.csv')
Tamagawa

,Tamagawa,arakawa,sumidagawa,edogawa,nakagawa,Tsurumigawa
0,5.626736,52.521197,31.512718,12.809319,12.809319,26.268151
1,5.825546,53.040814,31.824489,12.809319,12.809319,26.980355
2,5.825546,52.605087,31.563052,12.809319,12.809319,27.373281
3,6.050403,53.390313,32.034188,12.809319,12.809319,27.373281
4,6.050403,53.390313,32.034188,12.809319,12.809319,27.578950
...,...,...,...,...,...,...
8779,4.976854,47.644017,28.586410,5.123966,5.123966,24.180096
8780,4.958368,48.070388,28.842233,5.666074,5.666074,25.798441
8781,4.976854,47.644017,28.586410,6.209171,6.209171,26.611978
8782,4.976854,47.644017,28.586410,6.209171,6.209171,26.611978


In [24]:
#大芦橋と菅間の流量の和を荒川から東京湾への流入量とした